In [1]:
import numpy as np
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from sklearn.manifold.t_sne import _joint_probabilities
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
#from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
X, y = load_digits(return_X_y=True)
print(X.shape)

(1797, 64)


In [3]:
MACHINE_EPSILON = np.finfo(np.double).eps
n_components = 2
perplexity = 30

In [4]:
def fit(X):
    n_samples = X.shape[0]
    
    # Compute euclidean distance
    distances = pairwise_distances(X, metric='euclidean', squared=True)
    
    # Compute joint probabilities p_ij from distances.
    P = _joint_probabilities(distances=distances, desired_perplexity=perplexity, verbose=False)
    
    # The embedding is initialized with iid samples from Gaussians with standard deviation 1e-4.
    X_embedded = 1e-4 * np.random.mtrand._rand.randn(n_samples, n_components).astype(np.float32)
    
    # degrees_of_freedom = n_components - 1 comes from
    # "Learning a Parametric Embedding by Preserving Local Structure"
    # Laurens van der Maaten, 2009.
    degrees_of_freedom = max(n_components - 1, 1)
    
    return _tsne(P, degrees_of_freedom, n_samples, X_embedded=X_embedded)

In [5]:
def _tsne(P, degrees_of_freedom, n_samples, X_embedded):
    params = X_embedded.ravel()
    
    obj_func = _kl_divergence
    
    params = _gradient_descent(obj_func, params, [P, degrees_of_freedom, n_samples, n_components])
        
    X_embedded = params.reshape(n_samples, n_components)
    return X_embedded

In [6]:
x = np.array([[1, 2, 3], [4, 5, 6]])
np.ravel(x)

array([1, 2, 3, 4, 5, 6])

In [7]:
def _kl_divergence(params, P, degrees_of_freedom, n_samples, n_components):
    X_embedded = params.reshape(n_samples, n_components)
    
    dist = pdist(X_embedded, "sqeuclidean")
    dist /= degrees_of_freedom
    dist += 1.
    dist **= (degrees_of_freedom + 1.0) / -2.0
    Q = np.maximum(dist / (2.0 * np.sum(dist)), MACHINE_EPSILON)
    
    # Kullback-Leibler divergence of P and Q
    kl_divergence = 2.0 * np.dot(P, np.log(np.maximum(P, MACHINE_EPSILON) / Q))
    
    # Gradient: dC/dY
    grad = np.ndarray((n_samples, n_components), dtype=params.dtype)
    PQd = squareform((P - Q) * dist)
    for i in range(n_samples):
        grad[i] = np.dot(np.ravel(PQd[i], order='K'),
                         X_embedded[i] - X_embedded)
    grad = grad.ravel()
    c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
    grad *= c
    return kl_divergence, grad

In [8]:
def _gradient_descent(obj_func, p0, args, it=0, n_iter=1000,
                      n_iter_check=1, n_iter_without_progress=300,
                      momentum=0.8, learning_rate=200.0, min_gain=0.01,
                      min_grad_norm=1e-7):
    
    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = i = it
    
    for i in range(it, n_iter):
        error, grad = obj_func(p, *args)
        grad_norm = linalg.norm(grad)
        inc = update * grad < 0.0
        dec = np.invert(inc)
        gains[inc] += 0.2
        gains[dec] *= 0.8
        np.clip(gains, min_gain, np.inf, out=gains)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update
        print("[t-SNE] Iteration %d: error = %.7f, gradient norm = %.7f" % (i + 1, error, grad_norm))
        
        if error < best_error:
                best_error = error
                best_iter = i
        elif i - best_iter > n_iter_without_progress:
            break
        
        if grad_norm <= min_grad_norm:
            break
    return p

In [ ]:
X_embedded = fit(X)

[t-SNE] Iteration 1: error = 4.0229677, gradient norm = 0.0000049
[t-SNE] Iteration 2: error = 4.0229676, gradient norm = 0.0000049
[t-SNE] Iteration 3: error = 4.0229676, gradient norm = 0.0000059
[t-SNE] Iteration 4: error = 4.0229676, gradient norm = 0.0000090
[t-SNE] Iteration 5: error = 4.0229676, gradient norm = 0.0000151
[t-SNE] Iteration 6: error = 4.0229674, gradient norm = 0.0000273
[t-SNE] Iteration 7: error = 4.0229668, gradient norm = 0.0000527
[t-SNE] Iteration 8: error = 4.0229642, gradient norm = 0.0001074
[t-SNE] Iteration 9: error = 4.0229521, gradient norm = 0.0002310
[t-SNE] Iteration 10: error = 4.0228906, gradient norm = 0.0005219
[t-SNE] Iteration 11: error = 4.0225476, gradient norm = 0.0012329
[t-SNE] Iteration 12: error = 4.0204756, gradient norm = 0.0030108
[t-SNE] Iteration 13: error = 4.0074917, gradient norm = 0.0072111
[t-SNE] Iteration 14: error = 3.9373794, gradient norm = 0.0139920
[t-SNE] Iteration 15: error = 3.7143402, gradient norm = 0.0180028
[t-S

[t-SNE] Iteration 123: error = 0.9799667, gradient norm = 0.0006280
[t-SNE] Iteration 124: error = 0.9776389, gradient norm = 0.0005793
[t-SNE] Iteration 125: error = 0.9753392, gradient norm = 0.0005403
[t-SNE] Iteration 126: error = 0.9731378, gradient norm = 0.0005423
[t-SNE] Iteration 127: error = 0.9710415, gradient norm = 0.0005456
[t-SNE] Iteration 128: error = 0.9689851, gradient norm = 0.0005398
[t-SNE] Iteration 129: error = 0.9669448, gradient norm = 0.0005216
[t-SNE] Iteration 130: error = 0.9649320, gradient norm = 0.0005056
[t-SNE] Iteration 131: error = 0.9629580, gradient norm = 0.0004972
[t-SNE] Iteration 132: error = 0.9610216, gradient norm = 0.0004971
[t-SNE] Iteration 133: error = 0.9591208, gradient norm = 0.0004762
[t-SNE] Iteration 134: error = 0.9572643, gradient norm = 0.0004887
[t-SNE] Iteration 135: error = 0.9554302, gradient norm = 0.0004849
[t-SNE] Iteration 136: error = 0.9536152, gradient norm = 0.0004544
[t-SNE] Iteration 137: error = 0.9518220, gradie

[t-SNE] Iteration 244: error = 0.8362946, gradient norm = 0.0002803
[t-SNE] Iteration 245: error = 0.8355861, gradient norm = 0.0002587
[t-SNE] Iteration 246: error = 0.8348800, gradient norm = 0.0002731
[t-SNE] Iteration 247: error = 0.8341731, gradient norm = 0.0002692
[t-SNE] Iteration 248: error = 0.8334649, gradient norm = 0.0002631
[t-SNE] Iteration 249: error = 0.8327583, gradient norm = 0.0002569
[t-SNE] Iteration 250: error = 0.8320532, gradient norm = 0.0002488
[t-SNE] Iteration 251: error = 0.8313508, gradient norm = 0.0002753
[t-SNE] Iteration 252: error = 0.8306474, gradient norm = 0.0002600
[t-SNE] Iteration 253: error = 0.8299454, gradient norm = 0.0002682
[t-SNE] Iteration 254: error = 0.8292408, gradient norm = 0.0002539
[t-SNE] Iteration 255: error = 0.8285344, gradient norm = 0.0002534
[t-SNE] Iteration 256: error = 0.8278240, gradient norm = 0.0002595
[t-SNE] Iteration 257: error = 0.8271073, gradient norm = 0.0002740
[t-SNE] Iteration 258: error = 0.8263789, gradie

[t-SNE] Iteration 366: error = 0.7849772, gradient norm = 0.0001312
[t-SNE] Iteration 367: error = 0.7847723, gradient norm = 0.0001230
[t-SNE] Iteration 368: error = 0.7845702, gradient norm = 0.0001230
[t-SNE] Iteration 369: error = 0.7843699, gradient norm = 0.0001236
[t-SNE] Iteration 370: error = 0.7841707, gradient norm = 0.0001275
[t-SNE] Iteration 371: error = 0.7839722, gradient norm = 0.0001197
[t-SNE] Iteration 372: error = 0.7837747, gradient norm = 0.0001229
[t-SNE] Iteration 373: error = 0.7835777, gradient norm = 0.0001214
[t-SNE] Iteration 374: error = 0.7833819, gradient norm = 0.0001240
[t-SNE] Iteration 375: error = 0.7831884, gradient norm = 0.0001572
[t-SNE] Iteration 376: error = 0.7829977, gradient norm = 0.0002209
[t-SNE] Iteration 377: error = 0.7828084, gradient norm = 0.0002762
[t-SNE] Iteration 378: error = 0.7826147, gradient norm = 0.0002458
[t-SNE] Iteration 379: error = 0.7824186, gradient norm = 0.0001265
[t-SNE] Iteration 380: error = 0.7822307, gradie

[t-SNE] Iteration 486: error = 0.7634892, gradient norm = 0.0001289
[t-SNE] Iteration 487: error = 0.7633589, gradient norm = 0.0001062
[t-SNE] Iteration 488: error = 0.7632301, gradient norm = 0.0001396
[t-SNE] Iteration 489: error = 0.7630995, gradient norm = 0.0001120
[t-SNE] Iteration 490: error = 0.7629695, gradient norm = 0.0001145
[t-SNE] Iteration 491: error = 0.7628392, gradient norm = 0.0001192
[t-SNE] Iteration 492: error = 0.7627082, gradient norm = 0.0001087
[t-SNE] Iteration 493: error = 0.7625770, gradient norm = 0.0001109
[t-SNE] Iteration 494: error = 0.7624451, gradient norm = 0.0001143
[t-SNE] Iteration 495: error = 0.7623124, gradient norm = 0.0001063
[t-SNE] Iteration 496: error = 0.7621793, gradient norm = 0.0001128
[t-SNE] Iteration 497: error = 0.7620448, gradient norm = 0.0000994
[t-SNE] Iteration 498: error = 0.7619102, gradient norm = 0.0001160
[t-SNE] Iteration 499: error = 0.7617738, gradient norm = 0.0001079
[t-SNE] Iteration 500: error = 0.7616368, gradie

[t-SNE] Iteration 606: error = 0.7448616, gradient norm = 0.0000841
[t-SNE] Iteration 607: error = 0.7447765, gradient norm = 0.0000899
[t-SNE] Iteration 608: error = 0.7446907, gradient norm = 0.0000911
[t-SNE] Iteration 609: error = 0.7446042, gradient norm = 0.0000976
[t-SNE] Iteration 610: error = 0.7445168, gradient norm = 0.0001034
[t-SNE] Iteration 611: error = 0.7444280, gradient norm = 0.0000916
[t-SNE] Iteration 612: error = 0.7443385, gradient norm = 0.0000868
[t-SNE] Iteration 613: error = 0.7442478, gradient norm = 0.0000876
[t-SNE] Iteration 614: error = 0.7441557, gradient norm = 0.0000921
[t-SNE] Iteration 615: error = 0.7440617, gradient norm = 0.0000849
[t-SNE] Iteration 616: error = 0.7439661, gradient norm = 0.0000868
[t-SNE] Iteration 617: error = 0.7438685, gradient norm = 0.0000819
[t-SNE] Iteration 618: error = 0.7437692, gradient norm = 0.0000888
[t-SNE] Iteration 619: error = 0.7436673, gradient norm = 0.0000875
[t-SNE] Iteration 620: error = 0.7435627, gradie

[t-SNE] Iteration 727: error = 0.7267135, gradient norm = 0.0000798
[t-SNE] Iteration 728: error = 0.7266579, gradient norm = 0.0000823
[t-SNE] Iteration 729: error = 0.7266023, gradient norm = 0.0000780
[t-SNE] Iteration 730: error = 0.7265471, gradient norm = 0.0000797
[t-SNE] Iteration 731: error = 0.7264923, gradient norm = 0.0000808
[t-SNE] Iteration 732: error = 0.7264378, gradient norm = 0.0000825
[t-SNE] Iteration 733: error = 0.7263832, gradient norm = 0.0000657
[t-SNE] Iteration 734: error = 0.7263296, gradient norm = 0.0000750
[t-SNE] Iteration 735: error = 0.7262759, gradient norm = 0.0000729
[t-SNE] Iteration 736: error = 0.7262226, gradient norm = 0.0000672
[t-SNE] Iteration 737: error = 0.7261696, gradient norm = 0.0000683
[t-SNE] Iteration 738: error = 0.7261169, gradient norm = 0.0000685
[t-SNE] Iteration 739: error = 0.7260643, gradient norm = 0.0000649
[t-SNE] Iteration 740: error = 0.7260120, gradient norm = 0.0000653
[t-SNE] Iteration 741: error = 0.7259598, gradie

[t-SNE] Iteration 847: error = 0.7213865, gradient norm = 0.0000482
[t-SNE] Iteration 848: error = 0.7213528, gradient norm = 0.0000442
[t-SNE] Iteration 849: error = 0.7213193, gradient norm = 0.0000492
[t-SNE] Iteration 850: error = 0.7212858, gradient norm = 0.0000481
[t-SNE] Iteration 851: error = 0.7212525, gradient norm = 0.0000464
[t-SNE] Iteration 852: error = 0.7212193, gradient norm = 0.0000470
[t-SNE] Iteration 853: error = 0.7211862, gradient norm = 0.0000456
[t-SNE] Iteration 854: error = 0.7211532, gradient norm = 0.0000487
[t-SNE] Iteration 855: error = 0.7211204, gradient norm = 0.0000546
[t-SNE] Iteration 856: error = 0.7210877, gradient norm = 0.0000620
[t-SNE] Iteration 857: error = 0.7210549, gradient norm = 0.0000577
[t-SNE] Iteration 858: error = 0.7210222, gradient norm = 0.0000456
[t-SNE] Iteration 859: error = 0.7209897, gradient norm = 0.0000471
[t-SNE] Iteration 860: error = 0.7209574, gradient norm = 0.0000509
[t-SNE] Iteration 861: error = 0.7209250, gradie

[t-SNE] Iteration 968: error = 0.7179631, gradient norm = 0.0000394
[t-SNE] Iteration 969: error = 0.7179392, gradient norm = 0.0000398
[t-SNE] Iteration 970: error = 0.7179154, gradient norm = 0.0000389
[t-SNE] Iteration 971: error = 0.7178915, gradient norm = 0.0000383
[t-SNE] Iteration 972: error = 0.7178677, gradient norm = 0.0000381
[t-SNE] Iteration 973: error = 0.7178440, gradient norm = 0.0000382
[t-SNE] Iteration 974: error = 0.7178203, gradient norm = 0.0000383
[t-SNE] Iteration 975: error = 0.7177965, gradient norm = 0.0000381
[t-SNE] Iteration 976: error = 0.7177728, gradient norm = 0.0000372
[t-SNE] Iteration 977: error = 0.7177491, gradient norm = 0.0000391
[t-SNE] Iteration 978: error = 0.7177254, gradient norm = 0.0000401
[t-SNE] Iteration 979: error = 0.7177018, gradient norm = 0.0000418
[t-SNE] Iteration 980: error = 0.7176782, gradient norm = 0.0000461
[t-SNE] Iteration 981: error = 0.7176546, gradient norm = 0.0000487
[t-SNE] Iteration 982: error = 0.7176310, gradie

In [ ]:
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y, legend='full', palette=palette)